**Requirements
For Project 3, you will work with your group to tell a story using data visualizations. Here are the specific requirements:

Your visualization must include a Python Flask-powered API, HTML/CSS, JavaScript, and at least one database (SQL, MongoDB, SQLite, etc.).

Your project should fall into one of the following three tracks:

A combination of web scraping and Leaflet or Plotly

A dashboard page with multiple charts that update from the same data

A server that performs multiple manipulations on data in a database prior to visualization (must be approved)

Your project should include at least one JS library that we did not cover.

Your project must be powered by a dataset with at least 100 records.

Your project must include some level of user-driven interaction (e.g., menus, dropdowns, textboxes).

Your final visualization should ideally include at least three views.

When working on an online group project, it’s crucial to meet with your group and communicate regularly. Plan for significant collaboration time outside of class. The following tips can help you make the most of your time:

Decide how you’re going to communicate with your group members when you begin. Create a Slack channel, exchange phone numbers, and ensure that the group knows each group member’s available working hours.

Set up an agile project by using GitHub ProjectsLinks to an external site. so that your group can track tasks.

Create internal milestones to ensure that your group is on track. Set due dates for these milestones so that you have a timeline for completing the project. Some of these milestones might include:

Project ideation

Data fetching/API integration

Data analysis

Testing

Creating documentation

Creating the presentation


API call used in this project: Geoapify

In [34]:
 # Dependencies
import requests
import json
import pandas as pd
from config import geoapify_key
from splinter import Browser
from bs4 import BeautifulSoup as soup
from bs4 import BeautifulSoup
import re
import csv
from selenium import webdriver
import time
from urllib.parse import parse_qs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from urllib.parse import urlparse, parse_qs

In [35]:
# Set the geographical coordinates for Miami
latitude = 25.7617
longitude = -80.1918

# Set the parameters for the type of place
categories = "accommodation"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey": geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_data = response.json()


In [36]:
print(response.url)

https://api.geoapify.com/v2/places?categories=accommodation&limit=500&bias=proximity%3A-80.1918%2C25.7617&apiKey=be13e77afa4d4c778e3673d5943b0949


In [37]:
# Print the json (pretty printed)
#print(json.dumps(response_data, indent=4, sort_keys=True))



In [38]:
response_data['features'][0]['properties']

{'name': 'JW Marriot Miami',
 'country': 'United States',
 'country_code': 'us',
 'state': 'Florida',
 'county': 'Miami-Dade County',
 'city': 'Miami',
 'postcode': '33131',
 'street': 'Brickell Avenue',
 'housenumber': '1109',
 'lon': -80.19133949050796,
 'lat': 25.76246575,
 'state_code': 'FL',
 'formatted': 'JW Marriot Miami, 1109 Brickell Avenue, Miami, FL 33131, United States of America',
 'address_line1': 'JW Marriot Miami',
 'address_line2': '1109 Brickell Avenue, Miami, FL 33131, United States of America',
 'categories': ['accommodation',
  'accommodation.hotel',
  'building',
  'building.accommodation'],
 'details': ['details.building'],
 'datasource': {'sourcename': 'openstreetmap',
  'attribution': '© OpenStreetMap contributors',
  'license': 'Open Database Licence',
  'url': 'https://www.openstreetmap.org/copyright',
  'raw': {'name': 'JW Marriot Miami',
   'height': 127.7,
   'osm_id': 300247850,
   'tourism': 'hotel',
   'building': 'yes',
   'osm_type': 'w',
   'addr:cit

In [39]:
hotel_data = []
for feature in response_data['features']:
    # Use the get method to handle missing keys
    Hotel_Name = feature['properties'].get('name', None)
    city = feature['properties'].get('city', None)
    postcode = feature['properties'].get('postcode', None)
    housenumber = feature['properties'].get('housenumber', None)
    street = feature['properties'].get('street', None)
    state = feature['properties'].get('state', None)
    address2 = feature['properties'].get('address_line2', None)
    longitude= feature['properties'].get('lon', None)
    latitude= feature['properties'].get('lat', None)
    hotel_data.append([Hotel_Name, city, postcode, housenumber, street, state,  address2, longitude, latitude])

# Create a DataFrame from the extracted data
df = pd.DataFrame(hotel_data, columns=["Hotel Name", "city", "postcode", "housenumber", "street", "state","address2","longitude","latitude"])

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('./SQL_Tables/hotels.csv', index=False)


                                Hotel Name           city postcode  \
0                         JW Marriot Miami          Miami    33131   
1                        JW Marriott Miami          Miami    33131   
2             Hyatt Centric Brickell Miami          Miami    33131   
3                       SLS Brickell Miami          Miami    33131   
4             Hyatt Centric Brickell Miami          Miami    33131   
..                                     ...            ...      ...   
495  Four Points by Sheraton Miami Airport           None    33166   
496             Hampton Inn Miami Dadeland           None    33143   
497  WoodSpring Suites Doral Miami Airport           None    33166   
498                          Leeward Motel  Courtly Manor    33181   
499                                 Aida’s        Hialeah    33012   

    housenumber                 street    state  \
0          1109        Brickell Avenue  Florida   
1           111  Southeast 14th Street  Florida   
2     

In [40]:
hotel_address = []
for feature in response_data['features']:
    # Use the get method to handle missing keys
    city = feature['properties'].get('city', None)
    postcode = feature['properties'].get('postcode', None)
    housenumber = feature['properties'].get('housenumber', None)
    street = feature['properties'].get('street', None)
    state = feature['properties'].get('state', None)
    address2 = feature['properties'].get('address_line2', None)
    longitude= feature['properties'].get('lon', None)
    latitude= feature['properties'].get('lat', None)
    hotel_address.append([city, postcode, housenumber, street, state,  address2, longitude, latitude])

# Create a DataFrame from the extracted data
df = pd.DataFrame(hotel_address, columns=["city", "postcode", "housenumber", "street", "state","address2","longitude","latitude"])

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('./SQL_Tables/hotel_address.csv', index=False)


              city postcode housenumber                 street    state  \
0            Miami    33131        1109        Brickell Avenue  Florida   
1            Miami    33131         111  Southeast 14th Street  Florida   
2            Miami    33131        1102     Brickell Bay Drive  Florida   
3            Miami    33131        1300     South Miami Avenue  Florida   
4            Miami    33131        1101        Brickell Avenue  Florida   
..             ...      ...         ...                    ...      ...   
495           None    33166        3570  Northwest 74th Avenue  Florida   
496           None    33143        8200  Southwest 70th Avenue  Florida   
497           None    33166        7500  Northwest 41st Street  Florida   
498  Courtly Manor    33181       11790     Biscayne Boulevard  Florida   
499        Hialeah    33012         391       West 31st Street  Florida   

                                              address2  longitude   latitude  
0    1109 Brickell A

In [41]:
counter = 0

for place in response_data["features"]:
    print(f"hotel {counter+1}:")
    properties = place.get("properties", {})
    name = properties.get("name")
    city = properties.get("city")
    address_line1 = properties.get("address_line1")
    address_line2 = properties.get("address_line2")

    if name is not None:
        print(name)
    if city is not None:
        print(city)
    if address_line1 is not None:
        print(address_line1)
    if address_line2 is not None:
        print(address_line2)
        
    print("*"*20)
    counter = counter + 1
    if counter == 5:
        break


hotel 1:
JW Marriot Miami
Miami
JW Marriot Miami
1109 Brickell Avenue, Miami, FL 33131, United States of America
********************
hotel 2:
JW Marriott Miami
Miami
JW Marriott Miami
111 Southeast 14th Street, Miami, FL 33131, United States of America
********************
hotel 3:
Hyatt Centric Brickell Miami
Miami
Hyatt Centric Brickell Miami
1102 Brickell Bay Drive, Miami, FL 33131, United States of America
********************
hotel 4:
SLS Brickell Miami
Miami
SLS Brickell Miami
1300 South Miami Avenue, Miami, FL 33131, United States of America
********************
hotel 5:
Hyatt Centric Brickell Miami
Miami
Hyatt Centric Brickell Miami
1101 Brickell Avenue, Miami, FL 33131, United States of America
********************


In [42]:
# Set the geographical coordinates for Miami
latitude = 25.7617
longitude = -80.1918

# Set the parameters for the type of place
categories = "entertainment"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey": geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_ent = response.json()

In [43]:
# Print the json (pretty printed)
#print(json.dumps(response_ent, indent=4, sort_keys=True))

In [44]:
response_ent['features'][0]['properties']

{'name': 'CMX Brickell City Centre',
 'country': 'United States',
 'country_code': 'us',
 'state': 'Florida',
 'county': 'Miami-Dade County',
 'city': 'Miami',
 'postcode': '33130',
 'street': 'South Miami Avenue',
 'housenumber': '701',
 'lon': -80.1930845,
 'lat': 25.767149,
 'state_code': 'FL',
 'formatted': 'CMX Brickell City Centre, 701 South Miami Avenue, Miami, FL 33130, United States of America',
 'address_line1': 'CMX Brickell City Centre',
 'address_line2': '701 South Miami Avenue, Miami, FL 33130, United States of America',
 'categories': ['entertainment', 'entertainment.cinema'],
 'details': ['details.wiki_and_media'],
 'datasource': {'sourcename': 'openstreetmap',
  'attribution': '© OpenStreetMap contributors',
  'license': 'Open Database Licence',
  'url': 'https://www.openstreetmap.org/copyright',
  'raw': {'name': 'CMX Brickell City Centre',
   'osm_id': 6915346208,
   'screen': 10,
   'amenity': 'cinema',
   'osm_type': 'n',
   'wikidata': 'Q42421901',
   'addr:city':

In [45]:
data_ent = []

for feature in response_ent['features']:
    # Use the get method to handle missing keys
    entertainment = feature['properties'].get('name', None)
    city = feature['properties'].get('city', None)
    postcode = feature['properties'].get('postcode', None)
    housenumber = feature['properties'].get('housenumber', None)
    street = feature['properties'].get('street', None)
    state = feature['properties'].get('state', None)
    address2 = feature['properties'].get('address_line2', None)
    longitude= feature['properties'].get('lon', None)
    latitude= feature['properties'].get('lat', None)
    data_ent.append([entertainment, city, postcode, housenumber, street,state, address2,longitude,latitude ])

# Create a DataFrame from the extracted data
ent_df = pd.DataFrame(data_ent, columns=["entertainment", "city", "postcode","housenumber", "street", "state", "address2","longitude","latitude"])

# Display the DataFrame
print(ent_df)

# Save the DataFrame to a CSV file
ent_df.to_csv('./SQL_Tables/entertainment.csv', index=False)



                      entertainment          city postcode housenumber  \
0          CMX Brickell City Centre         Miami    33130         701   
1                Miami Lecture Hall         Miami    33131        None   
2                   Ashe Auditorium         Miami    33131        None   
3    James L. Knight Center Theater         Miami    33131        None   
4                 Silverspot Cinema         Miami    33131         300   
..                              ...           ...      ...         ...   
225              Historic Town Hall     Homestead    33030          41   
226                Seminole Theater     Homestead    33030          18   
227          Everglades Safari Park          None     None        None   
228     Claudia Castillo Art Studio          None    33305        None   
229          Florida Pioneer Museum  Florida City    33034         826   

                     street    state  \
0        South Miami Avenue  Florida   
1                 Riverwalk  Fl

In [54]:
ent_address = []

for feature in response_ent['features']:
    # Use the get method to handle missing keys
    city = feature['properties'].get('city', None)
    postcode = feature['properties'].get('postcode', None)
    housenumber = feature['properties'].get('housenumber', None)
    street = feature['properties'].get('street', None)
    state = feature['properties'].get('state', None)
    address2 = feature['properties'].get('address_line2', None)
    longitude= feature['properties'].get('lon', None)
    latitude= feature['properties'].get('lat', None)
    ent_address.append([city, postcode, housenumber, street,state, address2,longitude,latitude ])

# Create a DataFrame from the extracted data
entertainment_address = pd.DataFrame(ent_address, columns=["city", "postcode","housenumber", "street", "state", "address2","longitude", "latitude"])

# Display the DataFrame
print(entertainment_address)

# Save the DataFrame to a CSV file
ent_df.to_csv('./SQL_Tables/entertainment_address.csv', index=False)


             city postcode housenumber                  street    state  \
0           Miami    33130         701      South Miami Avenue  Florida   
1           Miami    33131        None               Riverwalk  Florida   
2           Miami    33131        None    Southeast 4th Street  Florida   
3           Miami    33131        None    Southeast 4th Street  Florida   
4           Miami    33131         300    Southeast 3rd Street  Florida   
..            ...      ...         ...                     ...      ...   
225     Homestead    33030          41  Southwest 177th Avenue  Florida   
226     Homestead    33030          18  Southwest 177th Avenue  Florida   
227          None     None        None    Southwest 8th Street  Florida   
228          None    33305        None            Wilton Drive  Florida   
229  Florida City    33034         826  Southwest 177th Avenue  Florida   

                                              address2  longitude   latitude  
0    701 South Miami

In [47]:
# Set the geographical coordinates for Miami
latitude = 25.7617
longitude = -80.1918

# Set the parameters for the type of place
categories = "public_transport"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey": geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_transport = response.json()

In [48]:
# Print the json (pretty printed)
#print(json.dumps(response_transport, indent=4, sort_keys=True))


In [49]:
response_transport['features'][0]['properties']

{'name': 'Brickell Avenue & Southeast 12th Street',
 'country': 'United States',
 'country_code': 'us',
 'state': 'Florida',
 'county': 'Miami-Dade County',
 'city': 'Miami',
 'postcode': '33131',
 'street': 'Brickell Avenue',
 'lon': -80.1916483,
 'lat': 25.762024,
 'state_code': 'FL',
 'formatted': 'Brickell Avenue & Southeast 12th Street, Brickell Avenue, Miami, FL 33131, United States of America',
 'address_line1': 'Brickell Avenue & Southeast 12th Street',
 'address_line2': 'Brickell Avenue, Miami, FL 33131, United States of America',
 'categories': ['public_transport', 'public_transport.bus'],
 'details': ['details'],
 'datasource': {'sourcename': 'openstreetmap',
  'attribution': '© OpenStreetMap contributors',
  'license': 'Open Database Licence',
  'url': 'https://www.openstreetmap.org/copyright',
  'raw': {'bin': 'yes',
   'bus': 'yes',
   'name': 'Brickell Avenue & Southeast 12th Street',
   'bench': 'yes',
   'osm_id': 1039660880,
   'ref:MT': '35;51',
   'highway': 'bus_st

In [50]:
trans_data = []

for feature in response_transport['features']:
    # Use the get method to handle missing keys
    name = feature['properties'].get('name', None)
    city = feature['properties'].get('city', None)
    postcode = feature['properties'].get('postcode', None)
    street = feature['properties'].get('street', None)
    address2 = feature['properties'].get('address_line2', None)
    Transport_Type = feature['properties']["datasource"]["raw"].get('operator', None)
    Transport_Operator= feature['properties']["datasource"]["raw"].get('operator', None)
    Transport=feature['properties']["datasource"]["raw"].get('highway', None)
    Route=feature['properties']["datasource"]["raw"].get('short_name', None)
    longitude= feature['properties'].get('lon', None)
    latitude= feature['properties'].get('lat', None)
    trans_data.append([name,city, postcode, street, address2,Transport_Type,Transport,Route,longitude,latitude])

                  
# Create a DataFrame from the extracted data
trans_data = pd.DataFrame(trans_data, columns=["name","city", "postcode", "street", "address2","Transport_Type","Transport","Route","longitude","latitude"])

# Drop rows with missing values
trans_data.dropna(inplace=True)

# Display the DataFrame
print(trans_data)

# Save the DataFrame to a CSV file
trans_data.to_csv('./SQL_Tables/trans_data.csv', index=False)


                                             name   city postcode  \
1         Brickell Avenue & Southeast 13th Street  Miami    33131   
2         Southeast 13th Street & Brickell Avenue  Miami    33131   
9        Brickell Avenue & Southeast 14th Terrace  Miami    33131   
25          Brickell Avenue & Southeast 15th Road  Miami    33131   
30          Brickell Avenue & Southeast 15th Road  Miami    33131   
..                                            ...    ...      ...   
464   Southwest 1st Street & Southwest 18th Court  Miami    33135   
465  Northwest 7th Street & Northwest 16th Avenue  Miami    33125   
469  Northwest 7th Street & Northwest 16th Avenue  Miami    33125   
497   Northwest 7th Street & Northwest 17th Court  Miami    33125   
499   Northwest 7th Street & Northwest 17th Court  Miami    33125   

                    street                                           address2  \
1          Brickell Avenue  Brickell Avenue, Miami, FL 33131, United State...   
2    Sout

In [51]:
trans_address = []

for feature in response_transport['features']:
    # Use the get method to handle missing keys
    city = feature['properties'].get('city', None)
    postcode = feature['properties'].get('postcode', None)
    street = feature['properties'].get('street', None)
    address2 = feature['properties'].get('address_line2', None)
    Transport_Type = feature['properties']["datasource"]["raw"].get('operator', None)
    Transport_Operator= feature['properties']["datasource"]["raw"].get('operator', None)
    Transport=feature['properties']["datasource"]["raw"].get('highway', None)
    Route=feature['properties']["datasource"]["raw"].get('short_name', None)
    longitude= feature['properties'].get('lon', None)
    latitude= feature['properties'].get('lat', None)
    trans_address.append([city, postcode, street, address2,Transport_Type,Transport,Route,longitude,latitude])

                  
# Create a DataFrame from the extracted data
trans_address = pd.DataFrame(trans_address, columns=["city", "postcode", "street", "address2","Transport_Type","Transport","Route","longitude","latitude"])

# Drop rows with missing values
trans_address.dropna(inplace=True)

# Display the DataFrame
print(trans_address)

# Save the DataFrame to a CSV file
trans_address.to_csv('./SQL_Tables/trans_address.csv', index=False)


      city postcode                 street  \
1    Miami    33131        Brickell Avenue   
2    Miami    33131  Southeast 13th Street   
9    Miami    33131        Brickell Avenue   
25   Miami    33131        Brickell Avenue   
30   Miami    33131        Brickell Avenue   
..     ...      ...                    ...   
464  Miami    33135   Southwest 1st Street   
465  Miami    33125   Northwest 7th Street   
469  Miami    33125   Northwest 7th Street   
497  Miami    33125   Northwest 7th Street   
499  Miami    33125   Northwest 7th Street   

                                              address2  \
1    Brickell Avenue, Miami, FL 33131, United State...   
2    Southeast 13th Street, Miami, FL 33131, United...   
9    Brickell Avenue, Miami, FL 33131, United State...   
25   Brickell Avenue, Miami, FL 33131, United State...   
30   Brickell Avenue, Miami, FL 33131, United State...   
..                                                 ...   
464  Southwest 1st Street, Miami, FL 3313